In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# pd.set_option('display.max_colwidth', None)

In [2]:
df = pd.read_csv('steam_games.csv')

try:
    os.remove('steam_games_processed.csv')
except:
    pass

In [3]:
df = df[df['name'].notna()]
df.drop(["discount_price", "publisher", "developer", "release_date", "desc_snippet", "types", "achievements", "recent_reviews", "mature_content"], axis=1, inplace=True)

In [ ]:
duplicated_items = df[df["name"].duplicated()]["name"].values.tolist()
games = {}
print(df.shape)
for row in df.iterrows():
    if row[1][1] in duplicated_items:
        if row[1][1] in games:
            games[row[1][1]].append(row[1].isna().sum())
        else:
            games[row[1][1]] = [row[1].isna().sum()]
for item in games:
    games[item].sort()
    count = 0
    for row in df.iterrows():
        if row[1][1] == item and (row[1].isnull().sum() == games[item][0]):
            df.drop(row[0], inplace=True)
            count += 1
            print("deleting: ", row[1][1])
        
        if count + 1 == len(games[item]):
            break
print(df.shape)

(40817, 11)
deleting:  RUSH
deleting:  Project Highrise
deleting:  Batman: Arkham City - Game of the Year Edition
deleting:  Angels of Death
deleting:  Luna
deleting:  Flashback
deleting:  New York Bus Simulator
deleting:  Warhammer Quest
deleting:  City Builder
deleting:  Killing Time
deleting:  Jigsaw Puzzles
deleting:  Hide and Seek
deleting:  Hidden Object Adventure Bundle
deleting:  Arcade Bundle
deleting:  Arcade Bundle
deleting:  Soundtrack Edition
deleting:  Solitaire Collection
deleting:  Ultimate Arena
deleting:  Black Rose
deleting:  Solstice
deleting:  Surge
deleting:  Game & Soundtrack
deleting:  7 Days in Dream 光尘
deleting:  Bounce
deleting:  Game + Soundtrack
deleting:  Chaos Theory
deleting:  Deluxe Edition
deleting:  Deluxe Edition
deleting:  Rumpus
deleting:  Space Maze
deleting:  Spark
deleting:  The Mine
deleting:  Memories
deleting:  Ashes
deleting:  Dodge
deleting:  We Were Here Too: Supporter Edition
deleting:  The Deer
deleting:  Zombie Apocalypse
deleting:  Aft

In [ ]:
all_review_cols = ["all_review_sentiment", "all_review_sentiment_num", "all_review_num"]
all_review_df = pd.DataFrame(columns=all_review_cols)
idx = 1
for row in df.iterrows():
    if row[1][3] == None or pd.isna(row[1][3]):
        all_review_df.loc[idx] = ["None", 0, 0]
    else:
        try:
            items = row[1][3].split('(')
            sentiment = items[0].lower().strip()[0:-1]
            items = items[1].split(')')
            total = int(items[0].strip().replace(',', ''))
            percent = int(items[1].split('%')[0].strip()[3:]) / 100
            all_review_df.loc[idx] = [sentiment, int(total * percent), total]
        except:
            all_review_df.loc[idx] = ["None", 0, 0]
    idx += 1
    if idx % 5000 == 0:
        print(((idx + 1) / 40752) * 100, "%")

In [ ]:
df.drop(["all_reviews"], axis=1, inplace=True)
df = pd.concat([df, all_review_df], axis=1)
df['all_review_sentiment'] = df['all_review_sentiment'].astype('category').cat.codes
df.to_csv("steam_games_processed.csv", index=False)